# [Module 2] 데이터 전처리

이 워크샵은 기본 커널을 conda_python3를 사용합니다.

## 0. 환경 설정

#### Library Import 

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 LAB을 위해서 핵심 Data Scientist용 Tool 인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [ ]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt

변수 로딩

In [ ]:
%store -r

## Personalize에서 학습 수행을 위한 데이터 제한 요건

Personalize에서 학습을 수행하기 위해서는 다음과 [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)같은 데이터 요구사항을 맞추어야 합니다. 

* 최소 25명 고유 사용자 
* 최소 100개 고유 아이템 
* 사용자 당 2개 이상의 Interaction(예. 구매,평가 등) 기록


하지만 일반적으로 다음과 같은 데이터가 준비 되어 있는것이 좋습니다. 

* 최소 50명 고유 사용자 
* 최소 100개 고유 아이템 
* 사용자 당 24 이상의 Interaction(예. 구매,평가 등) 기록

#### 중요 하이퍼 파라미터
**min_user_history_length_percentile, max_user_history_length_percentile**
- 일반적으로 가지고 있는 interaction의 데이타 셋이 사용자당 interaction의 수가 다릅니다.
즉 1개인 사용자도 많고, 엄청 많은 사용자도 있습니다. **이런 양 극단의 사용자를 제외하고 학습 데이타로 사용하는 것이 더욱 의미있는 추천을 받을 수 있습니다.** 아래 예 및 링크를 참고하시기 바랍니다.
    - For example, setting min_user_history_length_percentile to 0.05 and max_user_history_length_percentile to 0.95 includes all users except those with history lengths at the bottom or top 5%.


- User-Personalization Recipe
    - https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html

## 1. 데이터 탐색

### 1.1 전체 Interaction 분포

Interaction 분포는 아래와 같습니다. <br>

In [ ]:
pd.options.display.max_rows = 10
user_activity_counts = its.groupby("USER_ID").count().loc[:,["EVENT_TYPE"]].rename(columns={"EVENT_TYPE":"INTERACTION_COUNTS"})
user_activity_counts.describe()

### 1.2 Interaction 숫자 당 유저 수

In [ ]:
user_interaction_counts=user_activity_counts.reset_index()
activities = user_interaction_counts.groupby('INTERACTION_COUNTS').count()
activities.columns=['NUM_USERS']
activities

### 1.3 전체 Interactions 횟수 분포 확인

In [ ]:
num_interactions = 260
activities.loc[:num_interactions,:].plot(kind='bar', figsize=(15,5), ylim=(0,100))
plt.title("activities users group")

plt.show()

일부 분포 (100번)까지의 분포를 확인 합니다.

In [ ]:
num_interactions = 100
activities.loc[:num_interactions,:].plot(kind='bar', figsize=(15,5), ylim=(0,100))
plt.title("activities users group")

plt.show()

## 2. 데이터 필드 선택 및 변형

아래와 같은 순서로 데이터 필드를 선택하고 준비를 합니다.
- 필수적인 데이타를 중심으로 선택
- 퍼스널라이즈에서 요구하는 컬럼 이름으로 수정
- 선택된 데이터 필드의 타입을 적당하게 변경

### 참고
- ECOMMERCE datasets and schemas
    - https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-datasets-and-schemas.html

### 2.1 Items 컬럼 조정

In [ ]:
items.columns

In [ ]:
def item_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'ITEM_ID',
                              'name' : 'NAME',
                              'category' :'CATEGORY_L1',
                              'style' : 'STYLE',
                              'description' : 'PRODUCT_DESCRIPTION',
                              'price' : 'PRICE',
                              
                             })
    
    return ldf


item_cols = ['id', 'name', 'category', 'style', 'description','price']


items_df = item_data_selection(items, item_cols)    


items_df.head()

### 2.2 Users 컬럼 변경

In [ ]:
users.columns

In [ ]:
def user_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'USER_ID',
                              'username' : 'USER_NAME',
                              'age' :'AGE',
                              'gender' : 'GENDER',                              
                             })
    
    return ldf

user_cols = ['id', 'username', 'age', 'gender']


users_df = user_data_selection(users, user_cols)    
users_df.head()

### 2.3 Users 데이터 타입 변경

In [ ]:
users_df.info()

In [ ]:
def change_data_type(df, col, target_type):
    ldf = df.copy()
    ldf[col] = ldf[col].astype(target_type)
    
    return ldf

In [ ]:
users_df = change_data_type(users_df, col='USER_ID', target_type='object')
users_df.info()

### 2.4 Interactions 컬럼 변경

In [ ]:
its.columns

In [ ]:
def interactions_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'USER_ID',
                              'username' : 'USER_NAME',
                              'age' :'AGE',
                              'gender' : 'GENDER',                              
                             })
    
    return ldf

interactions_cols = ['ITEM_ID', 'USER_ID', 'EVENT_TYPE', 'TIMESTAMP']


full_interactions_df = interactions_data_selection(its, interactions_cols)    
full_interactions_df.head()

### 2.5 Interactions의 EVENT_TYPE의 두가지 선택

- 우리는 여기서 "ProductViewed" , "OrderCompleted" 를 선택하고 각각 이름을 "View" , "Purchase" 로 바꿉니다.

In [ ]:
full_interactions_df.EVENT_TYPE.value_counts()

In [ ]:
def filter_interactions_data(df, kinds_event_type):
    ldf = df[df['EVENT_TYPE'].isin(kinds_event_type)]
    ldf['EVENT_TYPE'] = ldf['EVENT_TYPE'].replace(['ProductViewed'],'View')    
    ldf['EVENT_TYPE'] = ldf['EVENT_TYPE'].replace(['OrderCompleted'],'Purchase')        
    
    return ldf

select_event_types = ['ProductViewed','OrderCompleted']
interactions_df = filter_interactions_data(full_interactions_df, select_event_types)
interactions_df

### 2.6 Interaction 데이터 타입 변경

In [ ]:
interactions_df.info()

In [ ]:
interactions_df = change_data_type(interactions_df, col='USER_ID', target_type='object')
interactions_df.info()

In [ ]:
interactions_df

## 3. 변수 저장

다음 노트북에서 활용할 변수를 저장 합니다.

In [ ]:
%store interactions_df
%store users_df
%store items_df